# Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Implement

In [2]:
class Elastic_Net_Regression:
    def __init__(self, alpha=0.0001, max_iter=1000, l1=0.01, l2=0.01):
        ''' 
        Class constructor
        
        Parameters
        ----------
        alpha: the learning rate determines how big the step would be on each iteration.
        max_iter: number of times update weight
        l1: l1 regularization
        l2: l2 regularization
        '''
        self.alpha = alpha
        self.max_iter = max_iter
        self.l1 = l1
        self.l2 = l2
        self.weight = None
        
    def fit(self, X, y):
        '''
        Trains Elastic Net Regression on the dataset (X, y).
        
        Parameters
        ----------
        X : numpy array, shape (m, n)
        The matrix of inputs
        y : numpy array, shape (m, 1) 
        The vector of outputs.
        '''
            
        # First column of this matrix is all ones (corresponding to x_0).
        X = np.append(np.ones((X.shape[0], 1)), X, axis=1)
        m, n = X.shape
        
        # Initialize weights shape (n + 1, 1)
        self.weight =  np.ones((n, 1))
        
        for iter in range(self.max_iter):
            error = (X @ self.weight) - y
            grad0 = X.T[0].dot(error) / m
            self.weight[0] -= self.alpha * grad0
            for i in range(1, len(self.weight[1:])):
                if self.weight[i] < 0:
                    grad = (X.T[i].dot(error) / m) - (self.l1 / m) + (self.l2 / m) * self.weight[i]
                    self.weight[i] -= self.alpha * grad
                if self.weight[i] > 0:
                    grad = (X.T[i].dot(error) / m) + (self.l1 / m) + (self.l2 / m) * self.weight[i]
                    self.weight[i] -= self.alpha * grad      
            
            #Early stopping
            if np.linalg.norm(self.weight, 2) < 1e-6:
                break     
                
    def predict(self, X):
        '''
        Predict using the ridge model.
        
        Parameters
        ----------
        X : numpy array, shape (m, n)
        The matrix of inputs
        
        Return
        ----------
        Returns predicted values.
        '''
        # First column of this matrix is all ones (corresponding to x_0).
        X = np.append(np.ones((X.shape[0], 1)), X, axis=1)
        return X @ self.weight

In [3]:
def standardScaler(X):
    return (X - np.mean(X)) / np.std(X)

In [4]:
def r2(y_test, y_pred):
    base = np.sum((y_pred - y_test.mean()) ** 2)
    mse = np.sum((y_pred - y_test) ** 2)
    r2 = 1 - (mse / base)
    return r2

# Test

In [5]:
#Read data
data = pd.read_excel('Data/Folds5x2_pp.xlsx').to_numpy()

In [6]:
X = data[:,:4]
y = data[:,-1].reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Using l1 = l2 = 0.1
model = Elastic_Net_Regression(alpha=0.4, max_iter=150000, l1=0.1)

X_train = standardScaler(X_train)
y_train = standardScaler(y_train)
model.fit(X_train, y_train)

X_test = standardScaler(X_test)
y_test = standardScaler(y_test)
y_pred = model.predict(X_test)

In [8]:
# Using l1 = l2 = 1
model = Elastic_Net_Regression(alpha=0.1, max_iter=150000, l1=1)

model.fit(X_train, y_train)

y_pred1 = model.predict(X_test)

In [9]:
# Using l1 = l2 = 5
model = Elastic_Net_Regression(alpha=0.1, max_iter=150000, l1=5)

model.fit(X_train, y_train)

y_pred2 = model.predict(X_test)

In [10]:
print('Score of using l1 = l2 = 0.1:', r2(y_test, y_pred))
print('Score of using l1 = l2 = 1:', r2(y_test, y_pred1))
print('Score of using l1 = l2 = 5:', r2(y_test, y_pred2))

Score of using l1 = l2 = 0.1: 0.9053963586456779
Score of using l1 = l2 = 1: 0.8704012711942936
Score of using l1 = l2 = 5: 0.8622711577404796
